In [122]:
import sys
import json
import random
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [126]:
from py2neo import Graph
from neo4j import GraphDatabase
NEO4J_URI="bolt://localhost:7687"
SC_JSON = "semantic-scholar-raw/s2-corpus-slim.json"

graph = Graph("localhost:7474")
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)

In [227]:
from faker import Faker
from faker.providers import lorem
fake = Faker()
fake.seed_instance(42)
fake.add_provider(lorem)

http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/corpus/

Download a part of the corpus with the `aws` command line interface, like

```aws s3 cp --no-sign-request s3://ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz .```

And then `gunzip` it.

In [31]:
!ls semantic-scholar-raw/

license.txt   s2-corpus-00	    s2-corpus-clean.tmp
manifest.txt  s2-corpus-clean.json  s2-corpus-slim.json


In [32]:
!wc -l semantic-scholar-raw/s2-corpus-00

1000000 semantic-scholar-raw/s2-corpus-00


Filter out those lines that have empty entities, empty inCitations, and empty outCitations

In [23]:
!grep -v -e '"entities":\[\]' -e '"inCitations":\[\]' -e '"outCitations":\[\]' semantic-scholar-raw/s2-corpus-00 > semantic-scholar-raw/s2-corpus-clean.tmp
!wc -l semantic-scholar-raw/s2-corpus-clean.tmp

212901 semantic-scholar-raw/s2-corpus-clean.tmp


Filter out those rows with empty attributes

In [28]:
!grep -v -e '"journalName":""' -e '"title":""' -e '"journalVolume":""' -e '"id":""' -e '"ids":""' -e '"doi":""' -e '"doiUrl":""' semantic-scholar-raw/s2-corpus-clean.tmp > semantic-scholar-raw/s2-corpus-clean.json
!wc -l semantic-scholar-raw/s2-corpus-clean.json

86609 semantic-scholar-raw/s2-corpus-clean.json


Obtain a smaller sample from that data

In [29]:
!head -100 semantic-scholar-raw/s2-corpus-clean.json > semantic-scholar-raw/s2-corpus-slim.json

In [27]:
!head -1 semantic-scholar-raw/s2-corpus-slim.json | jq

{
  "entities": [
    "Eating",
    "Ethanol",
    "Isopropyl Alcohol 0.7 ML/ML Medicated Pad"
  ],
  "journalVolume": "31 4",
  "journalPages": "385-8",
  "pmid": "8879286v1",
  "year": 1996,
  "outCitations": [
    "40bb6636803d10626bfafa3dbf940fdac903d11c",
    "18ffb4ae9e0d8f0c43342c4c6b993f3ce34bd579",
    "827b793fbc923d3e068b2008dc02bb63e7069fb9",
    "b96cb6dd0bba1e9928eb6315e18339f667c6799f",
    "48431576480666445bdcb112533aa885d4ac1ad1",
    "1380d901dbce2dca8eba90501179485d4fe1e5d4",
    "4203ad4dc8530a92473e5576650eca84b51c9eee",
    "96b85a405b002e11199f967280ec9a9736543c86",
    "8d72d01b1b670c7bb65408e91d1705f0c95b1174",
    "7842615744a118a2d4a239787e9450c26c98c9f9",
    "6ed7439f9537fce3f82144843622b9c670a95d7e",
    "55787878e91f6efdf14e16438e733d80581f3bec",
    "e28468ccfa0ddce1d060d39d25305f6abab2eced",
    "5f6ba13f126408d25849c518bddf4812281ae970"
  ],
  "s2Url": "https://semanticscholar.org/paper/30aaf92bde9bf6cdc3dff237691fd5db55d4cb4a",
  "s2PdfUrl": "http://

## Parse JSON
https://markhneedham.com/blog/2015/07/23/neo4j-loading-json-documents-with-cypher/

In [29]:
# Send Cypher query.
graph.run("RETURN apoc.version();").data()

[{'apoc.version()': '3.5.0.9'}]

In [28]:
graph.run("call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;").data()

[{'name': 'Neo4j Kernel', 'version': '3.5.15', 'edition': 'enterprise'}]

In [9]:
query = """
WITH "myjson" AS url 
CALL apoc.load.json(url) 
YIELD value 
RETURN value 
LIMIT 1"""

graph.run(query).data()

[{'value': {'doiUrl': '',
   'venue': 'Alcohol and alcoholism',
   'journalName': 'Alcohol and alcoholism',
   'sources': ['Medline'],
   'year': 1996,
   'paperAbstract': 'The present study was designed to assess the temporal pattern of ethanol intake over a 24 h period in selectively bred, Sardinian alcohol-preferring (sP) rats. Ethanol intake occurred under the two-bottle, free choice regimen. sP rats consumed ethanol in three distinct peaks, rather regularly distributed over the 12 h dark phase of the light-dark cycle and positively correlated with food intake episodes. The temporal distribution of ethanol intake and estimated blood alcohol levels are consistent with the hypothesis that sP rats voluntarily drink ethanol for its pharmacological effects.',
   'inCitations': ['7f17780491b74f08b3b6fb603014835e74ecfdd3',
    '9e2664ede0d21990f25b7682f023bc11e79e89b9',
    '1dbf8f6f64447b6f311031a14a7b99e43fb51f44',
    '842707e424ae0e2beda89c8847ce7c0fafa60b92',
    'a0711d2594d6e72cab3

Here, `"myjson"` is an alias for the path to the file, as defined in https://neo4j.com/docs/labs/apoc/current/import/load-json/

### create Article instances

In [168]:
query_apoc = """WITH "myjson" AS url 
CALL apoc.load.json(url) 
YIELD value as row
MERGE (art:Article {title: row.title, id: row.id, doi: row.doi, doi_url:row.doiUrl, year: row.year})
MERGE (jour:Journal {
    name: row.journalName,
    volume: row.journalVolume,
    pages: row.journalPages})-[:PUBLISHED_IN]->(art)
FOREACH (author in row.authors |
    MERGE (art)-[:AUTHORED_BY]->(auth:Author {name: author.name, id: author.ids})
)
FOREACH (k in row.entities |
    MERGE (art)-[:IS_ABOUT]->(kw: Keyword {name: k}))"""

with gdb.session() as session:
    session.run(query_apoc)

Some data is left to be simulated

### Faking citations

In [169]:
q_article_ids = "MATCH (n:Article) RETURN n.id"

with gdb.session() as session:
    article_ids = [v[0] for v in session.run(q_article_ids).values()]
    
article_ids[:5]
n_articles = len(article_ids)

In [170]:
# https://neo4j.com/docs/driver-manual/1.7/sessions-transactions/#driver-transactions-transaction-functions

q_add_citation_rel_id = """MATCH (a:Article),(b:Article)
WHERE a.id = $id_a AND b.id = $id_b
MERGE p=(a)-[r:CITED_BY]->(b)
RETURN p"""


def add_citation_rel(driver, id_a, id_b):
    with driver.session() as session:
        # Caller for transactional unit of work
        return session.write_transaction(create_citation_rel, id_a, id_b)

# Simple implementation of the unit of work
def create_citation_rel(tx, id_a, id_b):
    return tx.run(q_add_citation_rel_id, id_a=id_a, id_b = id_b)

Add 500 relationships of type `cited_by`

In [171]:
random.seed(42)
# pick a sample of 500 papers, and make them be cited by other three papers at random
created_citations = []

for article in random.sample(article_ids, round(n_articles*0.6)):
    for citation in random.sample(article_ids, 3):
        if article != citation: # they can't cite themselves
            _out = add_citation_rel(gdb, id_a=article, id_b=citation)
            created_citations.append(_out.data())
        else:
            print("Skipping: ",article, citation)

Skipping:  e06da9b34d6d1a7b57b27b5880c608c241adc5a2 e06da9b34d6d1a7b57b27b5880c608c241adc5a2
Skipping:  6f06076133abe3cd3bfed4d2f20c3fd97eef3702 6f06076133abe3cd3bfed4d2f20c3fd97eef3702
Skipping:  c104539460e4e4eac8b630ac81ef04d45a683286 c104539460e4e4eac8b630ac81ef04d45a683286
Skipping:  a57bf47fb69fc00e943ecf85e47444dbc5bee07a a57bf47fb69fc00e943ecf85e47444dbc5bee07a
Skipping:  2e277347fdc70287c56a4276d9d2e69055e3ecc8 2e277347fdc70287c56a4276d9d2e69055e3ecc8


### Faking reviewers 

We will create a `REVIEWED_BY` relationship between authors and articles

In [172]:
# delete relationships
with gdb.session() as session:
    _out = session.run("MATCH p=(:Article)-[r:REVIEWED_BY]->(:Author) DELETE r RETURN COUNT(r)")
    print(_out.data())

[{'COUNT(r)': 0}]


In [175]:
q_author_ids = "MATCH (n:Author) RETURN n.id"

with gdb.session() as session:
    author_ids = [v[0] for v in session.run(q_author_ids).values()]
    
author_ids[:5]

[['10399879'], ['1681558'], ['15577464'], ['11059246'], ['15538280']]

In [224]:
# assign between 3 and 4 reviewers randomly to 1000 articles
random.seed(42)

q_add_reviewers = """MATCH (a:Article),(b:Author)
WHERE a.id = $article_id AND b.id = $author_id
AND NOT (a)-[:AUTHORED_BY]->(b)
MERGE p=(a)-[:REVIEWED_BY]->(b)
RETURN p"""

created_review_rel = []


with gdb.session() as session:
    for article_id in random.sample(article_ids, 80):
        for author_id in random.sample(author_ids, random.choice([2, 4])):
            _out = session.run(q_add_reviewers, article_id=article_id, author_id=author_id)
            created_review_rel.append(_out.data())

In [208]:
created_review_rel[:2]

[[{'p': <Path start=<Node id=24791 labels={'Article'} properties={'doi_url': 'https://doi.org/10.3233/IKS-2012-0197', 'id': '0728b111bd17fda94b4861e79689edc01e492226', 'title': 'Causal inference and heterogeneity bias in social science', 'year': 2011, 'doi': '10.3233/IKS-2012-0197'}> end=<Node id=21528 labels={'Author'} properties={'name': 'Laura L Namy', 'id': ['2516786']}> size=1>}],
 [{'p': <Path start=<Node id=24791 labels={'Article'} properties={'doi_url': 'https://doi.org/10.3233/IKS-2012-0197', 'id': '0728b111bd17fda94b4861e79689edc01e492226', 'title': 'Causal inference and heterogeneity bias in social science', 'year': 2011, 'doi': '10.3233/IKS-2012-0197'}> end=<Node id=22052 labels={'Author'} properties={'name': 'Xizheng Zhang', 'id': ['39694885']}> size=1>}]]

### Fake conferences

In [297]:
# +-100 fake, non repeated keywords
fake_venues = (fake.company() for _ in range(100))
fake_conf_names = (fake.sentence(nb_words=3).rstrip('.') for _ in range(100))
fake_conf_edition = (random.randint(1,5) for _ in range(100))

fake_conferences = [
    (ix, name, venue, edition) for ix, (name, venue, edition) in enumerate(zip(fake_conf_names, fake_venues, fake_conf_edition), start=1)
]

In [298]:
print(len(fake_conferences))
fake_conferences[:10]

100


[(1, 'Action cut', 'Tyler and Sons', 4),
 (2, 'Nearly short box', 'Foster Group', 1),
 (3, 'Story', 'Bass Ltd', 3),
 (4, 'Trouble something population', 'Burton-Mcclain', 3),
 (5, 'Time leg sport', 'Brown, Meyer and Cunningham', 4),
 (6, 'New affect officer', 'Hendricks PLC', 4),
 (7, 'Wide exactly', 'Terrell Group', 5),
 (8, 'Number hope', 'Hancock-Walton', 5),
 (9, 'Report even beautiful', 'Watson, Gallegos and Turner', 5),
 (10, 'Few born size', 'Gentry, Wallace and Johnson', 2)]

In [289]:
# delete keywords
with gdb.session() as session:
    _out = session.run("MATCH (n:ConferenceProceedings) DELETE n RETURN COUNT(n)")
    print(_out.data())

[{'COUNT(n)': 0}]


In [304]:
q_create_conference = """MERGE (n:ConferenceProceedings {id:$id, name:$name, edition:$edition, venue:$venue})
RETURN n"""

created_conferences = []

with gdb.session() as session:
    for ix, name, venue, edition in fake_conferences:
        _out = session.run(q_create_conference, id=ix, name=name, edition=edition, venue=venue)
        created_conferences.append(_out.data())

In [326]:
# assign each paper to 3 conferences

q_link_to_conf = """MATCH (a:Article), (b:ConferenceProceedings)
WHERE a.id = $a_id AND b.id = $cp_id
MERGE p=(a)-[r:PUBLISHED_IN]->(b)
RETURN p"""
random.seed(42)
with gdb.session() as session:
    for article_id in random.sample(article_ids, round(n_articles*0.6)):
        for conf in random.sample(created_conferences, 3):
            _out = session.run(q_link_to_conf, a_id = article_id, cp_id = conf[0]["n"]["id"])

In [327]:
created_conferences[:5]

[[{'n': <Node id=10797 labels={'ConferenceProceedings'} properties={'name': 'Action cut', 'venue': 'Tyler and Sons', 'edition': 4, 'id': 1}>}],
 [{'n': <Node id=3276 labels={'ConferenceProceedings'} properties={'name': 'Nearly short box', 'venue': 'Foster Group', 'edition': 1, 'id': 2}>}],
 [{'n': <Node id=4867 labels={'ConferenceProceedings'} properties={'name': 'Story', 'venue': 'Bass Ltd', 'edition': 3, 'id': 3}>}],
 [{'n': <Node id=8202 labels={'ConferenceProceedings'} properties={'name': 'Trouble something population', 'venue': 'Burton-Mcclain', 'edition': 3, 'id': 4}>}],
 [{'n': <Node id=23456 labels={'ConferenceProceedings'} properties={'name': 'Time leg sport', 'venue': 'Brown, Meyer and Cunningham', 'edition': 4, 'id': 5}>}]]